# HW3.2: Neural Transition-Based Dependency Parsing


In this exercise, you are going to build a deep learning model for Neural Networks Transition-Based Dependency Parsing. A dependency parser analyzes the grammatical structure of a sentence, establishing relationships between “head” words and words which modify those heads. Your implementation will be a transition-based parser, which incrementally builds up a parse one step at a time.

To complete this exercise, you will need to complete the code and build a deep learning model for dependency parsing. 

We provide the code for data preparation and the skeleton for PartialParse class. You do not need to understand the code outside of this notebook. 


In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

In [2]:
# import shutil
# shutil.copy("/content/drive/MyDrive/FRA 501 IntroNLP&DL/Dataset/HW3-2.zip", "/content/HW3-2.zip")
# !unzip -q HW3-2.zip

## 1. Transition-Based Dependency Parsing

Your implementation will be a transition-based parser, which incrementally builds
up a parse one step at a time. At every step it maintains a partial parse, which is represented as follows:
- A stack of words that are currently being processed.
- A buffer of words yet to be processed.
- A list of dependencies predicted by the parser.

Initially, the stack only contains ROOT, the dependencies lists is empty, and the buffer contains all words
of the sentence in order. At each step, the parse applies a transition to the partial parse until its buffer is
empty and the stack is size 1. The following transitions can be applied:
- SHIFT: removes the first word from the buffer and pushes it onto the stack.
- LEFT-ARC: marks the second (second most recently added) item on the stack as a dependent of the
first item and removes the second item from the stack.
- RIGHT-ARC: marks the first (most recently added) item on the stack as a dependent of the second
item and removes the first item from the stack.

Your parser will decide among transitions at each state using a neural network classifier.

### TODO 1 (Written):
Go through the sequence of transitions needed for parsing the sentence “I parsed
this sentence correctly”. The dependency tree for the sentence is shown below. At each step, give the
configuration of the stack and buffer, as well as what transition was applied this step and what new
dependency was added (if any). The first three steps are provided below as an example.

Image --> https://drive.google.com/file/d/10jYgxDhsyolZGarcNTEdt6G2xB0l9iZU/view?usp=share_link  

![TODO 1 IMG](pic/HW3-2_img1.jpg)

Complete the following table (double click the table and fill in the rest):

| stack                             |  buffer                                   |  new dependency   | transition            |
| :------:                          | :------:                                  | :-------------:   | :--------:            |
| \[ROOT\]                          | \[I, parsed, this, sentence, correctly\]  |                   | Initial Configuration |
| \[ROOT, I\]                       | \[parsed, this, sentence, correctly\]     |                   | SHIFT                 |
| \[ROOT, I, parsed\]               | \[this, sentence, correctly\]             |                   | SHIFT                 |
| \[ROOT, parsed\]                  | \[this, sentence, correctly\]             | parsed→I          | LEFT-ARC              |
| \[ROOT, parsed, this\]            | \[sentence, correctly\]                   |                   | SHIFT                 |
| \[ROOT, parsed, this, sentence\]  | \[correctly\]                             |                   | SHIFT                 |
| \[ROOT, parsed, sentence\]        | \[correctly\]                             | sentence→this     | LEFT-ARC              |
| \[ROOT, parsed\]                  | \[correctly\]                             | parsed→sentence   | RIGHT-ARC             |
| \[ROOT, parsed, correctly\]       | \[\]                                      |                   | SHIFT                 |
| \[ROOT, parsed\]                  | \[\]                                      | parsed→correctly  | RIGHT-ARC             |
| \[ROOT\]                          | \[\]                                      | ROOT→parsed       | RIGHT-ARC             |

References:
- https://medium.com/mlearning-ai/dependency-parsing-with-neural-networks-e36f5166628d
- https://www.emnlp2014.org/papers/pdf/EMNLP2014082.pdf

### TODO 2 (Coding):
Implement the __\_\_init\_\___ and __parse_step__ functions in the PartialParse class. Your code must past both of the following tests.

In [3]:
class PartialParse(object):
    def __init__(self, sentence):
        """Initializes this partial parse.

        Your code should initialize the following fields:
            self.stack: The current stack represented as a list with the top of the stack as the
                        last element of the list.
            self.buffer: The current buffer represented as a list with the first item on the
                         buffer as the first item of the list
            self.dependencies: The list of dependencies produced so far. Represented as a list of
                    tuples where each tuple is of the form (head, dependent).
                    Order for this list doesn't matter.

        The root token should be represented with the string "ROOT"

        Args:
            sentence: The sentence to be parsed as a list of words.
                      Your code should not modify the sentence.
        """
        # The sentence being parsed is kept for bookkeeping purposes. Do not use it in your code.
        self.sentence = sentence #--list

        ### YOUR CODE HERE
        self.stack = ['ROOT']  # --> list
        self.buffer = self.sentence.copy() # --> list
        self.dependencies = []  # --> list
        ### END YOUR CODE

    def parse_step(self, transition):
        """Performs a single parse step by applying the given transition to this partial parse

        Args:
            transition: A string that equals "S", "LA", or "RA" representing the shift, left-arc,
                        and right-arc transitions. You can assume the provided transition is a legal
                        transition.
        """
        ### YOUR CODE HERE
        if transition == 'S':
            self.stack.append(self.buffer.pop(0))
        if transition == 'LA':
            self.dependencies.append((self.stack[-1], self.stack[-2]))
            self.stack.pop(-2)
        if transition == 'RA':
            self.dependencies.append((self.stack[-2], self.stack[-1]))
            self.stack.pop(-1)
        ### END YOUR CODE

    def parse(self, transitions):
        """Applies the provided transitions to this PartialParse

        Args:
            transitions: The list of transitions in the order they should be applied
        Returns:
            dependencies: The list of dependencies produced when parsing the sentence. Represented
                          as a list of tuples where each tuple is of the form (head, dependent)
        """
        for transition in transitions:
            self.parse_step(transition)
        return self.dependencies

In [4]:
# Do not modify this code
def test_step(name, transition, stack, buf, deps,
              ex_stack, ex_buf, ex_deps):
    """Tests that a single parse step returns the expected output"""
    pp = PartialParse([])
    pp.stack, pp.buffer, pp.dependencies = stack, buf, deps

    pp.parse_step(transition)
    stack, buf, deps = (tuple(pp.stack), tuple(pp.buffer), tuple(sorted(pp.dependencies)))
    assert stack == ex_stack, \
        "{:} test resulted in stack {:}, expected {:}".format(name, stack, ex_stack)
    assert buf == ex_buf, \
        "{:} test resulted in buffer {:}, expected {:}".format(name, buf, ex_buf)
    assert deps == ex_deps, \
        "{:} test resulted in dependency list {:}, expected {:}".format(name, deps, ex_deps)
    print("{:} test passed!".format(name))


def test_parse_step():
    """Simple tests for the PartialParse.parse_step function
    Warning: these are not exhaustive
    """
    test_step("SHIFT", "S", ["ROOT", "the"], ["cat", "sat"], [],
              ("ROOT", "the", "cat"), ("sat",), ())
    test_step("LEFT-ARC", "LA", ["ROOT", "the", "cat"], ["sat"], [],
              ("ROOT", "cat",), ("sat",), (("cat", "the"),))
    test_step("RIGHT-ARC", "RA", ["ROOT", "run", "fast"], [], [],
              ("ROOT", "run",), (), (("run", "fast"),))


def test_parse():
    """Simple tests for the PartialParse.parse function
    Warning: these are not exhaustive
    """
    sentence = ["parse", "this", "sentence"]
    dependencies = PartialParse(sentence).parse(["S", "S", "S", "LA", "RA", "RA"])
    dependencies = tuple(sorted(dependencies))
    expected = (('ROOT', 'parse'), ('parse', 'sentence'), ('sentence', 'this'))
    assert dependencies == expected,  \
        "parse test resulted in dependencies {:}, expected {:}".format(dependencies, expected)
    assert tuple(sentence) == ("parse", "this", "sentence"), \
        "parse test failed: the input sentence should not be modified"
    print("parse test passed!")

In [5]:
test_parse_step()
test_parse()

SHIFT test passed!
LEFT-ARC test passed!
RIGHT-ARC test passed!
parse test passed!


## 2. Setup and Preprocessing

In [6]:
from utils.parser_utils import minibatches, load_and_preprocess_data

Preparing data. We will use a subset of Penn Treebank and pretrained embeddings in this task

We are now going to train a neural network to predict, given the state of the stack, buffer, and dependencies, which transition should be applied next. First, the model extracts a feature vector representing the current state. We will be using the feature set presented in the original neural dependency parsing paper: A Fast and Accurate Dependency Parser using Neural Networks. 

The function extracting these features has been implemented for you in parser_utils. This feature vector consists of a list of tokens (e.g., the last word in the stack, first word in the buffer, dependent of the second-to-last word in the stack if there is one, etc.). They can be represented as a list of integers.

In [7]:
parser, embeddings, train_examples, dev_set, test_set = load_and_preprocess_data(True)

Loading data...
took 1.96 seconds
Building parser...
took 0.02 seconds
Loading pretrained embeddings...
took 2.23 seconds
Vectorizing data...
took 0.08 seconds
Preprocessing training data...
took 1.15 seconds


In [8]:
print(len(train_examples), len(dev_set), len(test_set))

48390 500 500


In [9]:
train_examples[10]

([5156,
  660,
  88,
  96,
  85,
  2131,
  5155,
  5155,
  5155,
  5155,
  5155,
  5155,
  91,
  5155,
  113,
  5155,
  5155,
  5155,
  84,
  39,
  40,
  61,
  41,
  39,
  83,
  83,
  83,
  83,
  83,
  83,
  40,
  83,
  41,
  83,
  83,
  83],
 [1, 1, 1],
 2)

In [10]:
embeddings

array([[-0.35588855,  0.6878136 , -0.27288294, ...,  1.2894735 ,
        -0.8015917 , -0.832532  ],
       [ 1.5229279 , -0.14771435,  0.64237845, ...,  0.1900125 ,
         0.5338304 , -0.9380984 ],
       [-2.1680155 , -0.7834329 ,  0.47289106, ..., -0.74451214,
        -1.5166008 ,  0.02513922],
       ...,
       [ 0.50952065, -0.4281435 ,  0.51560694, ..., -0.83727574,
         0.581582  ,  2.9018507 ],
       [-1.3396579 ,  0.47578225,  0.8213765 , ..., -0.39435554,
         0.35004184,  1.0678409 ],
       [-1.016677  ,  0.6115748 , -0.4053425 , ..., -1.1876915 ,
         0.02993344, -0.4450842 ]], dtype=float32)

In [12]:
print(embeddings.shape)

(5157, 50)


Get the full batch of our subset data

In [13]:
minibatch_gen = minibatches(train_examples, len(train_examples))
x_train, y_train = minibatch_gen.__next__()

In [14]:
print(x_train.shape)
print(y_train.shape)

(48390, 36)
(48390, 3)


In [15]:
x_train[0]

array([ 111,  113,  578,   88,  140, 1239, 1746, 5155, 5155, 5155, 5155,
       5155, 5155, 5155, 5155, 5155, 5155, 5155,   40,   41,   39,   40,
         40,   49,   43,   83,   83,   83,   83,   83,   83,   83,   83,
         83,   83,   83])

You can use parser.id2tok[word_id] to lookup the word in English.

In [16]:
for word_id in x_train[0]:
  print(parser.id2tok[word_id])

with
an
offer
of
about
900
initial
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<NULL>
<p>:IN
<p>:DT
<p>:NN
<p>:IN
<p>:IN
<p>:CD
<p>:JJ
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>
<p>:<NULL>


In [17]:
y_train[0]

array([1., 0., 0.])

## 3. Model

In [194]:
from tensorflow.keras.models import Sequential, Model, load_model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input, Dense, Dropout, Flatten
from tensorflow.keras.optimizers import Adam
from kerastuner.tuners import RandomSearch
from tensorflow.random import set_seed
set_seed(2)

### TODO 3 (Coding):
Build and train a tensroflow keras model to predict an action for each state of of the input. This is a simple classification task. 
- The input and output of the model must match the dimention of x_train and y_train.
- The model must use the provided pretrained embeddings
- The model could comprise of only a feedforward layer and a dropout
- Training loss should be around 0.1 or below, and training categorical_accuracy above 0.94

In [70]:
# model = Sequential()
# # Write your code here
# model.add(Embedding(input_dim=embeddings.shape[0], output_dim=embeddings.shape[1], weights=[embeddings], trainable=False))
# model.add(Dense(128, activation='relu'))
# model.add(Dropout(0.5))
# model.add(Dense(3, activation='softmax'))
# model.add(Flatten())
# model.add(Reshape((3,)))

# model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])

In [87]:
x_train.shape

(48390, 36)

In [88]:
y_train.shape

(48390, 3)

In [89]:
embeddings.shape

(5157, 50)

In [167]:
# def create_model(embeddings, x_shape, y_shape):
#     model = Sequential()
#     model.add(Input(shape=(x_shape,)))
#     model.add(Embedding(input_dim=embeddings.shape[0], output_dim=embeddings.shape[1], trainable=False))
#     model.add(Dense(128, activation='relu'))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(64, activation='relu'))
#     model.add(Dense(32, activation='relu'))
#     model.add(Dropout(0.5))
#     model.add(Flatten())
#     model.add(Dense(y_shape, activation='softmax'))
#     # model.add(Reshape((y_shape,)))
#     model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
#     return model

# model = create_model(embeddings, x_train.shape[1], y_train.shape[1])
# model.summary()

Model: "sequential_40"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_52 (Embedding)    (None, 36, 50)            257850    
                                                                 
 dense_85 (Dense)            (None, 36, 128)           6528      
                                                                 
 dense_86 (Dense)            (None, 36, 64)            8256      
                                                                 
 dense_87 (Dense)            (None, 36, 64)            4160      
                                                                 
 dense_88 (Dense)            (None, 36, 32)            2080      
                                                                 
 dropout_36 (Dropout)        (None, 36, 32)            0         
                                                                 
 flatten_19 (Flatten)        (None, 1152)            

In [176]:
def createModel_tuner(hp):
    embeddings = (5157, 50)
    x_shape = 36
    y_shape = 3
    model = Sequential()
    model.add(Input(shape=(x_shape,)))
    model.add(Embedding(input_dim=embeddings[0], output_dim=embeddings[1], trainable=False))
    for i in range(hp.Int('num_layers', 2, 10)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                        min_value=32,
                                        max_value=256,
                                        step=32),
                            activation='relu'))
    model.add(Dropout(hp.Float('dropout', 0, 0.8, step=0.1, default=0.5)))
    model.add(Flatten())
    model.add(Dense(y_shape, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=0.001), metrics=['categorical_accuracy'])
    return model

In [182]:
tuner = RandomSearch(
    createModel_tuner,
    objective='categorical_accuracy',
    max_trials=10,
    directory='tunerDir',
    project_name='tuner')

tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 10, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': 'linear'}
dropout (Float)
{'default': 0.5, 'conditions': [], 'min_value': 0.0, 'max_value': 0.8, 'step': 0.1, 'sampling': 'linear'}


In [183]:
tunerHistory = tuner.search(x_train, y_train, epochs=70, batch_size=32, validation_split=0.2)

Trial 10 Complete [00h 19m 47s]
categorical_accuracy: 0.9711975455284119

Best categorical_accuracy So Far: 0.9857150316238403
Total elapsed time: 02h 37m 20s
INFO:tensorflow:Oracle triggered exit


In [186]:
tuner.results_summary(num_trials=5)

Results summary
Results in tunerDir\tuner
Showing 5 best trials
Objective(name="categorical_accuracy", direction="max")

Trial 06 summary
Hyperparameters:
num_layers: 6
units_0: 64
units_1: 192
dropout: 0.0
units_2: 256
units_3: 32
units_4: 224
units_5: 160
units_6: 192
units_7: 160
Score: 0.9857150316238403

Trial 08 summary
Hyperparameters:
num_layers: 4
units_0: 96
units_1: 224
dropout: 0.30000000000000004
units_2: 224
units_3: 96
units_4: 160
units_5: 192
units_6: 64
units_7: 160
units_8: 160
units_9: 32
Score: 0.9750723242759705

Trial 09 summary
Hyperparameters:
num_layers: 8
units_0: 256
units_1: 192
dropout: 0.1
units_2: 32
units_3: 160
units_4: 32
units_5: 256
units_6: 64
units_7: 256
units_8: 128
units_9: 32
Score: 0.9711975455284119

Trial 01 summary
Hyperparameters:
num_layers: 8
units_0: 192
units_1: 32
dropout: 0.0
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
Score: 0.9672194719314575

Trial 05 summary
Hyperparameters:
num_layers: 7
units_0: 64


In [199]:
# tunedModel = tuner.get_best_models(num_models=1)[0]
# tunedModel.save('hw3_2_best_model')

model = load_model('hw3_2_best_model')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 36, 50)            257850    
                                                                 
 dense (Dense)               (None, 36, 64)            3264      
                                                                 
 dense_1 (Dense)             (None, 36, 192)           12480     
                                                                 
 dense_2 (Dense)             (None, 36, 256)           49408     
                                                                 
 dense_3 (Dense)             (None, 36, 32)            8224      
                                                                 
 dense_4 (Dense)             (None, 36, 224)           7392      
                                                                 
 dense_5 (Dense)             (None, 36, 160)           3

In [200]:
# Write your code here
model.fit(x_train, y_train, epochs=70, batch_size=32, use_multiprocessing=True)

Epoch 1/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.1308 - categorical_accuracy: 0.9631
Epoch 2/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.1008 - categorical_accuracy: 0.9665
Epoch 3/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0923 - categorical_accuracy: 0.9693
Epoch 4/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0869 - categorical_accuracy: 0.9699
Epoch 5/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0821 - categorical_accuracy: 0.9718
Epoch 6/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0803 - categorical_accuracy: 0.9712
Epoch 7/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0729 - categorical_accuracy: 0.9743
Epoch 8/70
1513/1513 [==============================] - 13s 9ms/step - loss: 0.0708 - categorical_accuracy: 0.9751
Epoch 9/70
1513/1513 [==============================] - 14s 9ms/step - loss: 0.0